In [ ]:
using Plots, LaTeXStrings
using STMO

![](Figures/logo.png)

# Motivation

Up to now, we confidently assumed that we would always be able to compute the derivative or gradient of any function. Despite differentiation being a relatively easy operation, it is frequenty not feasible (or desirable) to compute this by hand. *Numerical differentiation* can provide approximations of th derivate or gradient at a particular point. *Automatic differentiation* directly manipulates the computational graph to generate a function that computes the (exact) derivate. Such methods have advanced greatly in the last years and it is no exageration that their easy use in popular software libraries such as TenserFlow and PyTorch are a cornerstone of deep learning and other machine learning and scientific computing fields.

# Definition of a derivative

In [ ]:
using Plots, BenchmarkTools
using STMO

# Definition of a derivative

$$
\frac{\text{d}f(x)}{\text{d}x} = f'(x) = \lim _{h\to 0}{\frac {f(x+h)-f(x)}{h}}.
$$

Derivation is in essence a mechanical process, following the rules below.

![](Figures/derivatives.jpeg)

When we work with function of several variables, we use *partial derivatives* (e.g. $\frac{\partial f(x, y)}{\partial x}$), indicating we keep all variables but $x$ fixed.

Our running example:

$$
f(x) = \log x + \frac{\sin x}{x}
$$

In [ ]:
f(x) = log(x) + sin(x) / x;

# Symbolic differentiation

Computing derivatives, as you have seen in basic calculus courses.

By hand or automatically:
- Maple
- Sympy (python)
- Mathematica
- Maxima

Differentiation is *easy* compared to *integration* or *sampling*.

Advantages:
- exact derivatives!
- gives the formula for different evaluations.
    - insight in the system
    - in some cases, closed-form solution extrema by solving $\frac{\text{d}f(x)}{\text{d}x}=0$
- no hyperparameters or tweaking: just works!

Disadvantages:
- some software not flexible enough (gradients, arrays, for-loops,...)
- sometimes explosion of terms: *expression swell*
- not always numerically optimal!

In [ ]:
using SymEngine

In [ ]:
@vars x  # define variable

In [ ]:
df = diff(f(x), x)

In [ ]:
df(2.0)

In [ ]:
plot(f, 1, 5, label="\$f(x)\$", xlabel="\$x\$", lw=2, color=mygreen)
plot!(df, 1, 5, label="\$f'(x)\$", lw=2, color=myorange)

# Numerical differentiation

Finite difference approximation of the derivative/gradient based on a number of function evaluations.

Often based on the limit definition of a derivative. Theoretical analysis using Taylor approximation:

$$
f(x + h) = f(x) + \frac{h}{1!}f'(x) + \frac{h^2}{2!}f''(x) + \frac{h^3}{3!}f^{(3)}(x)+\ldots
$$

**Forward difference**

$$
f'(x)\approx \frac{f(x+h) - f(x)}{h}
$$

**Central difference**

$$
f'(x)\approx \frac{f(x+h) - f(x-h)}{2h}
$$

**Complex step method**

$$
f'(x)\approx \frac{\text{Im}(f(x +ih))}{h}
$$

In [ ]:
diff_fordiff(f, x; h=1e-10) = (f(x + h) - f(x)) / h;
diff_centrdiff(f, x; h=1e-10) = (f(x + h) - f(x - h)) / 2h;
diff_complstep(f, x; h=1e-10) = imag(f(x + im * h)) / h;

In [ ]:
diff_fordiff(f, 2.0)

In [ ]:
diff_centrdiff(f, 2.0)

In [ ]:
diff_complstep(f, 2.0)

## Intermezzo: floats

Real numbers are always represented as floating point numbers in a computer.

![Encoding of a real number using a `Float32`.](Figures/floats.png)

By default, Julia uses double precision floats (`Float64`). For brevity, let us take a look at the bit representation of a float. We use `Float32` for brevity's sake.

In [ ]:
num = Float32(10.789)
bitstring(num)

The first bit encodes the *sign*, here positive.

In [ ]:
sign(num)

The next eight bits specify the *exponent*, the magnitude of the number.

In [ ]:
exponent(num)

While the final 23 bits specify the *mantissa*, a number between $[1,2]$ representing the precision.

In [ ]:
significand(num)

These can be used to reconstrunct the number.

In [ ]:
significand(num) * 2^exponent(num)

The *machine precision* of a number can be retained using `eps`. This is the relative error.

In [ ]:
eps(num)  # not very high because it is only Float32

This means that larger numbers have a larger absolute error compared to small numbers.

In [ ]:
eps(1.2)

In [ ]:
eps(1.2e10)

In [ ]:
eps(1.2e-10)

This brings us with numerical issues we might encounter using numerical differentiation.

**First sin of numerical analysis**:

> *thou shalt not add small numbers to big numbers*

**second sin of numerical analysis**:

> *thou shalt not subtract numbers which are approximately equal*

## Back to numerical differentiation

In [ ]:
fexamp(x) = 64x*(1-x)*(1-2x)^2*(1-8x+8x^2)^2
dfexamp = diff(fexamp(x), x)
error(diff, h; x=1.0) = max(abs(Float64(dfexamp(x)) - diff(fexamp, x, h=h)), 1e-50);

In [ ]:
stepsizes = map(t->10.0^t, -20:0.1:-1);
plot(stepsizes, error.(diff_fordiff, stepsizes), label="forward difference",
    xscale=:log10, yscale=:log10, lw=2, legend=:bottomright, color=myblue)
plot!(stepsizes, error.(diff_centrdiff, stepsizes), label="central difference", lw=2,
            color=myred)
plot!(stepsizes, error.(diff_complstep, stepsizes), label="complex step", lw=2,
            color=myyellow)
#xlims!(1e-15, 1e-1)
xlabel!("\$h\$")
ylabel!("absolute error")

Advantages of numerical differentiation:
- easy to implement
- general, no assumptions needed

Disadvantages:
- not numerically stable (round-off errors)
- not efficient for gradients ($\mathcal{O}(n)$ evaluations for $n$-dimensional vectors)


## Approximations of multiplications with gradients

**Gradient-vector approximation**

$$
\nabla f(\mathbf{x})^\intercal \mathbf{d} \approx \frac{f(\mathbf{x}+h\cdot\mathbf{d}) - f(\mathbf{x}-h\cdot\mathbf{d})}{2h}
$$

**Hessian-vector approximation**

$$
\nabla^2 f(\mathbf{x}) \mathbf{d} \approx \frac{\nabla f(\mathbf{x}+h\cdot\mathbf{d}) - \nabla f(\mathbf{x}-h\cdot\mathbf{d})}{2h}
$$

In [ ]:
grad_vect(f, x, d; h=1e-10) = (f(x + h * d) - f(x - h * d)) / (2h)

In [ ]:
dvect = randn(10) / 10
xvect = 2rand(10)

A = randn(10, 10)
A = A * A' / 100

#g(x) = exp(- x' * A * x)  # adjoint does not play with Zygote
g(x) = exp(- sum(x .* (A * x)))

# correct gradient and Hessian (by hand)
Dg(x) = -2g(x) * A * x
D²g(x) = -2g(x) * A - 2A * x * Dg(x)'

In [ ]:
g(xvect)

In [ ]:
Dg(xvect)

In [ ]:
Dg(xvect)' * dvect

In [ ]:
grad_vect(g, xvect, dvect)

In [ ]:
D²g(xvect) * dvect

In [ ]:
h = 1e-10
(Dg(xvect + h * dvect) - Dg(xvect - h * dvect)) / 2h

# Forward differentiation

Accumulation of the gradients along the *computational graph*.

<img src="Figures/compgraph.png" alt="drawing" width="400"/>

Forward differentiation computes the gradient from the inputs to the outputs.

## Differentiation rules

**Sum rule**:

$$
\frac{\partial (f(x)+g(x))}{\partial x} =  \frac{\partial f(x)}{\partial x} + \frac{\partial f(x)}{\partial x}
$$

**Product rule**:

$$
\frac{\partial (f(x)g(x))}{\partial x} =  f(x)\frac{\partial g(x)}{\partial x} + g(x)\frac{\partial f(x)}{\partial x}
$$

**Chain rule**:

$$
\frac{\partial (g(f(x))}{\partial x} =  \frac{\partial g(u)}{\partial u}\mid_{u=f(x)} \frac{\partial f(x)}{\partial x}
$$

## Example of the forward differentiation

<img src="Figures/forwarddiff.png" alt="drawing" width="600"/>

## Dual numbers

Forward differentiation can be viewed as evaluating function using *dual numbers*, which can be viewed as truncated Taylor series:

$$
v + \dot{v}\epsilon\,,
$$

where $v,\dot{v}\in\mathbb{R}$ and $\epsilon$ a nilpotent number, i.e. $\epsilon^2=0$. For example, we have

$$
(v + \dot{v}\epsilon) + (u + \dot{u}\epsilon) = (v+u) + (\dot{v} +\dot{u})\epsilon
$$


$$
(v + \dot{v}\epsilon)(u + \dot{u}\epsilon) = (vu) + (v\dot{u} +\dot{v}u)\epsilon\,.
$$


These dual numbers can be used as

$$
f(v+\dot{v}\epsilon) = f(v) + f'(v)\dot{v}\epsilon\,.
$$

In [ ]:
struct Dual{T}
    v::T
    vdot::T
end

Let's implement some basic rules showing linearity.

In [ ]:
Base.:+(a::Dual, b::Dual) = Dual(a.v + b.v, a.vdot + b.vdot)
Base.:*(a::Dual, b::Dual) = Dual(a.v * b.v, a.v * b.vdot + b.v * a.vdot)
Base.:+(c::Real, b::Dual) = Dual(c + b.v, b.vdot)
Base.:*(v::Real, b::Dual) = Dual(v, 0.0) * b

And some more advanced ones, based on differentiation.

In [ ]:
Base.:sin(a::Dual) = Dual(sin(a.v), cos(a.v) * a.vdot)
Base.:exp(a::Dual) = Dual(exp(a.v), exp(a.v) * a.vdot)
Base.:log(a::Dual) = Dual(log(a.v), 1.0 / a.v * a.vdot)
Base.:/(a::Dual, b::Dual) = Dual(a.v / b.v, (a.vdot * b.v - a.v * b.vdot) / b.v^2)

In [ ]:
f(Dual(2.0, 1.0))

In [ ]:
myforwarddiff(f, x) = f(Dual(x, 1.0)).vdot

myforwarddiff(f, 2.0)

This directly works for vectors!

In [ ]:
q(x) = 10.0 * x[1] * x[2] + x[1] * x[1] + sin(x[1]) / x[2]

In [ ]:
q([1, 2])

In [ ]:
q(Dual.([1, 2], [1, 0]))  # partial wrt x1

In [ ]:
q(Dual.([1, 2], [0, 1]))  # partial wrt x2

In practice, we prefer to use a package to do this.

In [ ]:
using ForwardDiff

In [ ]:
ForwardDiff.derivative(f, 2.0)

In [ ]:
ForwardDiff.gradient(g, xvect)

In [ ]:
ForwardDiff.gradient(q, [1, 2])

Forward differentiation:

- exact gradients!
- computational complexity scales with **number of inputs**
- used when you have more outputs than inputs

# Reverse differentiation

Compute the gradient from the output toward the inputs using the chain rule.

<img src="Figures/reversediff.png" alt="drawing" width="600"/>

Reverse differentiation:

- also exact!
- main workhorse for training artificial neural networks.
- efficient when more inputs than outputs (machine learning: thousands of parameters vs. one loss)

In [ ]:
using Zygote

In [ ]:
f'(2.0)  # that's it

Works as well:

In [ ]:
Zygote.gradient(f, 2.0)

Fuctions with more than one variable.

In [ ]:
g'(xvect)

Finding the Hessian:

In [ ]:
Zygote.hessian(g, xvect)

## Artificial neural networks

Multi-layer perceptron.

<img src="Figures/ANN_example.png" alt="drawing" width="200"/>

Forward differentiation.

<img src="Figures/Forwardprop.png" alt="drawing" width="500"/>


Reverse differentation or backpropagation.

<img src="Figures/Backprop.png" alt="drawing" width="500"/>

Returns effect of changing layer output on the loss. Can be related directly to the parameters!

## Exercise: logistic regression

Recall logistic regression on a training set $S=\{(\mathbf{x}_i, y_i)\mid i=1,\ldots,n\}$ with $y\in\{0,1\}$.

Prediction:

$$
f(\mathbf{x}) = \sigma(\mathbf{w}^\intercal\mathbf{x})\,,
$$

with $\sigma(t) = 1 /(1+exp(t))$.

To find the parameter vector $\mathbf{w}$, we minimize the cross-entropy:

$$
L(\mathbf{w};S)= \sum_{i=1}^n = - y_i \log(f(\mathbf{x})) - (1-y_i)\log(1-f(\mathbf{x}))\,.
$$

In [ ]:
# artificial data
X = [randn(50, 2); randn(50, 2) .+ [-1.0 2.4]];
y = [i <= 50 ? 0 : 1 for i in 1:100];
n = length(y);

scatter(X[:,1], X[:,2], color=y)

In [ ]:
σ(t) = 1.0 / (1.0 + exp(t))
f(x, w) = σ(sum(x .* w))
L(w; X=X, y=y) = sum(- y .* log.(σ.(X * w)) - (1.0 .- y) .* log.(1. .- σ.( X * w)))

In [ ]:
w = [0.1, 0.1]
L(w)

**Assignments**

1. Compute the gradient of $L$ w.r.t. $\mathbf{w}$ using
    - numerical method
    - forward differentiation
    - backward differentiation
2. (optional) Implement a simple gradient descent to find  $\mathbf{w}^\star$.
3. Add a bias to the prediction function. Use `Zygote` to compute the gradients w.r.t. both parameters.

# Differentiating ODE

Automatic differentiation can be used beyond machine learning and optimization:

- [physical engines](https://arxiv.org/abs/1611.01652) to learn robot control
- differentiating [protein](https://github.com/lupoglaz/TorchProteinLibrary) [structures](https://www.cell.com/cell-systems/fulltext/S2405-4712(19)30076-6)
- Sinkhorn algorithm
- [dynamic programming](https://arxiv.org/abs/1802.03676)
- [differential equations](https://julialang.org/blog/2019/01/fluxdiffeq)

Everything is computed by some straightforward and differentiable functions!

# Exercise

Consider the *Wheeler's Ridge* function:

$$
f(\mathbf{x}) = -\exp(-(x_1 x_2 - a)^2 -(x_2 -a)^2)\,,
$$

at the point $\mathbf{x}_0=[1.5, 1.5]^T$. We set $a=1.5$.

Implement this function.

Compute the gradient by hand.

Find the gradient and Hessian at $\mathbf{x}_0$ by numerical differentiation.

Compute the gradient and Hessian at $\mathbf{x}_0$ using automatic differentiation.

# References

- Gunes et. al. (2015) *Automatic differentiation in machine learning: a survey*
- Kochenderfer, M. J. and Wheeler, T., '*Algorithms for Optimization*'. MIT Press (2019)